<a href="https://colab.research.google.com/github/JKourelis/Colab_Helixer/blob/main/Helixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/weberlab-hhu/Helixer/main/img/helixer.svg" height="200" align="right" style="height:240px">

# Helixer: Deep Learning Gene Annotation

**Structural genome annotation using Deep Neural Networks and Hidden Markov Models**

Helixer is a tool for _ab initio_ prediction of gene structure that directly provides primary gene models in GFF3 format. It utilizes Deep Neural Networks to identify which base pairs belong to UTR/CDS/Intron regions of genes, making it performant and applicable to a wide variety of genomes.

**Key Features:**
- **Four trained lineage models**: fungi, land_plant, vertebrate, and invertebrate
- **Intelligent parameter scaling**: Auto-adjusts settings based on genome assembly quality
- **High performance**: Optimized for GPU acceleration with realistic genome-scale datasets
- **Direct output**: Produces GFF3 files ready for downstream analysis
- **Effector annotation**: Enhanced sensitivity with adjustable peak thresholds
- **Sequence extraction**: Automatic protein, transcript, CDS, and gene sequence extraction
- **Geneious compatibility**: Creates viewer-compatible GFF files
- **Genome quality assessment**: N50/N90 analysis with assembly quality classification

**Supported Genomes:**
- Minimum sequence length: 25 kbp per record
- Maximum recommended file size: 1 GB
- Supports compressed FASTA files (.gz, .zip, .bz2)
- Automatically optimizes parameters for assembly quality

Felix Holst, Anthony Bolger, Christopher Günther, Janina Maß, Sebastian Triesch, Felicitas Kindel, Niklas Kiel, Nima Saadat, Oliver Ebenhöh, Björn Usadel, Rainer Schwacke, Marie Bolger, Andreas P.M. Weber, Alisandra K. Denton. Helixer—de novo Prediction of Primary Eukaryotic Gene Models. 2023. *bioRxiv*, [10.1101/2023.02.06.527280](https://doi.org/10.1101/2023.02.06.527280)

Felix Stiehler, Marvin Steinborn, Stephan Scholz, Daniela Dey, Andreas P M Weber, Alisandra K Denton. 2020. Helixer: Cross-species gene annotation of large eukaryotic genomes using deep learning. *Bioinformatics*, [btaa1044](https://doi.org/10.1093/bioinformatics/btaa1044)

In [ ]:
# CELL 2: Installation and Setup
#@title Install Helixer and Dependencies
#@markdown This cell installs all required dependencies for Helixer including system packages, Python libraries, gffread, and the HelixerPost processor.

import subprocess
import sys
import os
import shutil

def run_command(cmd, description):
    """Run command with error handling and progress reporting."""
    print(f"[{description}]")
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"❌ FAILED: {result.stderr[:500]}")
        return False
    print("✅ OK")
    return True

def install_helixer():
    """Complete Helixer installation process."""

    # Update system packages
    if not run_command("apt-get update -qq", "Updating system packages"):
        return False

    # Install system dependencies including Rust and gffread
    if not run_command("apt-get install -y python3-dev build-essential git wget curl gffread", "Installing system dependencies"):
        return False

    # Install Rust (required for HelixerPost)
    print("[Installing Rust for HelixerPost]")
    if not os.path.exists("/root/.cargo/bin/cargo"):
        if not run_command("curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y", "Installing Rust"):
            return False
        # Add Rust to PATH
        os.environ['PATH'] = f"/root/.cargo/bin:{os.environ['PATH']}"
    else:
        print("✅ Rust already installed")

    # Clone Helixer repository
    if not os.path.exists("/content/Helixer"):
        if not run_command("cd /content && git clone https://github.com/weberlab-hhu/Helixer.git", "Cloning Helixer repository"):
            return False
    else:
        print("✅ Helixer repository already exists")

    # Clone and install HelixerPost
    if not os.path.exists("/content/HelixerPost"):
        if not run_command("cd /content && git clone https://github.com/TonyBolger/HelixerPost.git", "Cloning HelixerPost repository"):
            return False
    else:
        print("✅ HelixerPost repository already exists")

    # Build HelixerPost with Cargo
    if not run_command("cd /content/HelixerPost && /root/.cargo/bin/cargo build --release", "Building HelixerPost with Rust"):
        return False

    # Install HelixerPost binary
    helixer_post_src = "/content/HelixerPost/target/release/helixer_post_bin"
    if os.path.exists(helixer_post_src):
        if not run_command(f"cp {helixer_post_src} /usr/local/bin/", "Installing HelixerPost binary"):
            return False
    else:
        print("❌ HelixerPost binary not found after build")
        return False

    # Upgrade pip and install wheel
    if not run_command(f"{sys.executable} -m pip install --upgrade pip wheel", "Upgrading pip and installing wheel"):
        return False

    # Install Python requirements
    if not run_command(f"cd /content/Helixer && {sys.executable} -m pip install -r requirements.3.10.txt", "Installing Python requirements"):
        return False

    # Install Helixer
    if not run_command(f"cd /content/Helixer && {sys.executable} -m pip install .", "Installing Helixer"):
        return False

    # Install TensorFlow with GPU support
    if not run_command(f"{sys.executable} -m pip install tensorflow[and-cuda]", "Installing TensorFlow with GPU support"):
        return False

    # Verify GPU availability
    print("[Verifying GPU support]")
    try:
        import tensorflow as tf
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"✅ GPU detected: {len(gpus)} device(s)")
            for gpu in gpus:
                print(f"   - {gpu}")
        else:
            print("⚠️  No GPU detected - predictions will be slow")
    except Exception as e:
        print(f"⚠️  GPU verification error: {e}")

    # Add Helixer scripts to PATH
    helixer_scripts = "/content/Helixer"
    if helixer_scripts not in os.environ.get('PATH', ''):
        os.environ['PATH'] = f"{helixer_scripts}:{os.environ['PATH']}"

    print("\n🎉 Helixer installation completed successfully!")
    return True

# Execute installation
if install_helixer():
    print("\n📋 Installation Summary:")
    print("✅ System dependencies installed")
    print("✅ gffread installed for sequence extraction")
    print("✅ Helixer repository cloned")
    print("✅ HelixerPost processor built and installed")
    print("✅ Python requirements installed")
    print("✅ Helixer package installed")
    print("✅ TensorFlow with GPU support installed")
    print("\n🚀 Ready to run Helixer predictions!")
else:
    print("\n❌ Installation failed. Please check error messages above.")

[Updating system packages]
✅ OK
[Installing system dependencies]
✅ OK
[Installing Rust for HelixerPost]
[Installing Rust]
✅ OK
[Cloning Helixer repository]
✅ OK
[Cloning HelixerPost repository]
✅ OK
[Building HelixerPost with Rust]
✅ OK
[Installing HelixerPost binary]
✅ OK
[Upgrading pip and installing wheel]
✅ OK
[Installing Python requirements]
✅ OK
[Installing Helixer]
✅ OK
[Installing TensorFlow with GPU support]
✅ OK
[Verifying GPU support]
⚠️  GPU verification error: maximum recursion depth exceeded while calling a Python object

🎉 Helixer installation completed successfully!

📋 Installation Summary:
✅ System dependencies installed
✅ gffread installed for sequence extraction
✅ Helixer repository cloned
✅ HelixerPost processor built and installed
✅ Python requirements installed
✅ Helixer package installed
✅ TensorFlow with GPU support installed

🚀 Ready to run Helixer predictions!


In [ ]:
# CELL 3: Google Drive Setup and File Input Configuration
#@title Setup Google Drive and Configure Input Files
#@markdown Authenticate with Google Drive, specify input genome location, and configure output folder.

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.colab import files
import gzip
import zipfile
import bz2
import io
import re

# Input method selection
input_method = "Google Drive Path" #@param ["Upload File", "Google Drive Path"]
#@markdown Choose whether to upload a file or use an existing file in Google Drive

# Google Drive file path (only used if "Google Drive Path" selected)
gdrive_file_path = "/Fusarium/GCA_001702875.1_Fol004_genomic.fasta.gz" #@param {type:"string"}
#@markdown Google Drive path to genome file. Format: /folder/subfolder/filename.fasta

# Output folder configuration
output_folder_name = "Helixer_Output" #@param {type:"string"}
#@markdown Name of the output folder in Google Drive root

# Global variables
drive_service = None
helixer_folder_id = None
input_fasta_path = None
input_species_name = None
genome_stats = None

def calculate_genome_statistics(fasta_path):
    """Calculate comprehensive genome assembly statistics including N50/N90."""
    print("📊 Calculating genome assembly statistics...")

    # Read all sequences and their lengths
    sequence_lengths = []
    total_length = 0
    total_ungapped_length = 0
    total_n_count = 0
    total_gc_count = 0
    total_at_count = 0
    sequence_count = 0

    with open(fasta_path, 'r') as f:
        current_seq_length = 0
        current_seq = ""

        for line in f:
            line = line.strip()
            if line.startswith('>'):
                # Process previous sequence
                if current_seq:
                    sequence_lengths.append(current_seq_length)
                    total_length += current_seq_length

                    # Count Ns, GC, AT
                    n_count = current_seq.upper().count('N')
                    gc_count = current_seq.upper().count('G') + current_seq.upper().count('C')
                    at_count = current_seq.upper().count('A') + current_seq.upper().count('T')

                    total_n_count += n_count
                    total_gc_count += gc_count
                    total_at_count += at_count
                    total_ungapped_length += (current_seq_length - n_count)
                    sequence_count += 1

                # Reset for new sequence
                current_seq_length = 0
                current_seq = ""
            else:
                current_seq_length += len(line)
                current_seq += line

        # Process last sequence
        if current_seq:
            sequence_lengths.append(current_seq_length)
            total_length += current_seq_length

            n_count = current_seq.upper().count('N')
            gc_count = current_seq.upper().count('G') + current_seq.upper().count('C')
            at_count = current_seq.upper().count('A') + current_seq.upper().count('T')

            total_n_count += n_count
            total_gc_count += gc_count
            total_at_count += at_count
            total_ungapped_length += (current_seq_length - n_count)
            sequence_count += 1

    # Sort sequences by length (largest first) for N50/N90 calculation
    sequence_lengths.sort(reverse=True)

    # Calculate N50 and N90
    def calculate_nx(lengths, percentage):
        target_length = sum(lengths) * (percentage / 100.0)
        cumulative_length = 0
        lx_count = 0

        for length in lengths:
            cumulative_length += length
            lx_count += 1
            if cumulative_length >= target_length:
                return length, lx_count
        return lengths[-1] if lengths else 0, len(lengths)

    n50, l50 = calculate_nx(sequence_lengths, 50)
    n90, l90 = calculate_nx(sequence_lengths, 90)

    # Calculate percentages
    gc_percent = (total_gc_count / (total_gc_count + total_at_count)) * 100 if (total_gc_count + total_at_count) > 0 else 0
    gap_percent = (total_n_count / total_length) * 100 if total_length > 0 else 0

    # Classify assembly quality
    if n50 >= 1_000_000:  # >= 1 Mb
        quality_class = "Excellent"
        quality_desc = "Chromosome-level"
        quality_emoji = "🏆"
    elif n50 >= 100_000:  # >= 100 kb
        quality_class = "Good"
        quality_desc = "Scaffold-level"
        quality_emoji = "🥈"
    elif n50 >= 10_000:   # >= 10 kb
        quality_class = "Fair"
        quality_desc = "Contig-level"
        quality_emoji = "🥉"
    else:
        quality_class = "Poor"
        quality_desc = "Fragmented"
        quality_emoji = "⚠️"

    # Create statistics dictionary
    stats = {
        'total_length': total_length,
        'ungapped_length': total_ungapped_length,
        'sequence_count': sequence_count,
        'n50': n50,
        'n90': n90,
        'l50': l50,
        'l90': l90,
        'longest_sequence': max(sequence_lengths) if sequence_lengths else 0,
        'shortest_sequence': min(sequence_lengths) if sequence_lengths else 0,
        'average_length': total_length / sequence_count if sequence_count > 0 else 0,
        'median_length': sequence_lengths[len(sequence_lengths)//2] if sequence_lengths else 0,
        'gc_percent': gc_percent,
        'gap_percent': gap_percent,
        'quality_class': quality_class,
        'quality_desc': quality_desc,
        'quality_emoji': quality_emoji
    }

    return stats

def display_genome_statistics(stats):
    """Display formatted genome statistics."""
    print(f"\n📊 Genome Assembly Statistics:")
    print(f"═══════════════════════════════")
    print(f"{stats['quality_emoji']} Assembly Quality: {stats['quality_class']} ({stats['quality_desc']})")
    print(f"📏 Genome size: {stats['total_length'] / 1_000_000:.1f} Mb")
    print(f"📐 Ungapped length: {stats['ungapped_length'] / 1_000_000:.1f} Mb ({100 - stats['gap_percent']:.1f}%)")
    print(f"🧩 Number of sequences: {stats['sequence_count']:,}")
    print(f"📊 Sequence N50: {stats['n50'] / 1000:.1f} kb")
    print(f"📊 Sequence N90: {stats['n90'] / 1000:.1f} kb")
    print(f"🔢 Sequence L50: {stats['l50']:,}")
    print(f"🔢 Sequence L90: {stats['l90']:,}")
    print(f"📏 Longest sequence: {stats['longest_sequence'] / 1000:.1f} kb")
    print(f"📏 Average sequence: {stats['average_length'] / 1000:.1f} kb")
    print(f"🧪 GC content: {stats['gc_percent']:.1f}%")
    if stats['gap_percent'] > 0:
        print(f"🕳️  Gap content: {stats['gap_percent']:.1f}%")

def setup_google_drive():
    """Setup Google Drive authentication and output folder."""
    global drive_service, helixer_folder_id

    print("🔐 Authenticating with Google Drive...")
    try:
        auth.authenticate_user()
        drive_service = build('drive', 'v3')
        print("✅ Google Drive authentication successful")

        # Create or find output folder
        print(f"📁 Setting up output folder: {output_folder_name}")

        # Search for existing folder
        query = f"name='{output_folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
        results = drive_service.files().list(q=query).execute()
        folders = results.get('files', [])

        if folders:
            helixer_folder_id = folders[0]['id']
            print(f"✅ Found existing folder: {output_folder_name}")
        else:
            # Create new folder
            folder_metadata = {
                'name': output_folder_name,
                'mimeType': 'application/vnd.google-apps.folder'
            }
            folder = drive_service.files().create(body=folder_metadata).execute()
            helixer_folder_id = folder['id']
            print(f"✅ Created new folder: {output_folder_name}")

        return True

    except Exception as e:
        print(f"❌ Google Drive setup failed: {e}")
        return False

def find_file_by_path(path):
    """Find file in Google Drive by path."""
    if not drive_service:
        return None

    # Clean and split path
    path = path.strip().strip('/')
    if not path:
        return None

    path_parts = path.split('/')
    filename = path_parts[-1]
    folder_path = path_parts[:-1] if len(path_parts) > 1 else []

    print(f"🔍 Searching for: {filename}")
    if folder_path:
        print(f"   📁 In folder path: /{'/'.join(folder_path)}")

    try:
        # Navigate through folder structure
        current_parent = 'root'

        for folder_name in folder_path:
            query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and '{current_parent}' in parents and trashed=false"
            results = drive_service.files().list(q=query).execute()
            folders = results.get('files', [])

            if not folders:
                print(f"❌ Folder not found: {folder_name}")
                return None

            current_parent = folders[0]['id']
            print(f"   ✅ Found folder: {folder_name}")

        # Search for file in final folder
        query = f"name='{filename}' and '{current_parent}' in parents and trashed=false"
        results = drive_service.files().list(q=query).execute()
        files_found = results.get('files', [])

        if not files_found:
            print(f"❌ File not found: {filename}")
            return None

        if len(files_found) > 1:
            print(f"⚠️  Multiple files found with name: {filename}")
            print("   Using the first one found")

        file_info = files_found[0]
        print(f"✅ Found file: {file_info['name']} (ID: {file_info['id']})")
        return file_info

    except Exception as e:
        print(f"❌ Error searching for file: {e}")
        return None

def download_from_gdrive(file_info):
    """Download file from Google Drive."""
    try:
        print(f"⬇️  Downloading {file_info['name']} from Google Drive...")

        request = drive_service.files().get_media(fileId=file_info['id'])
        downloaded_content = io.BytesIO()
        downloader = MediaIoBaseDownload(downloaded_content, request)

        done = False
        while done is False:
            status, done = downloader.next_chunk()
            if status:
                progress = int(status.progress() * 100)
                if progress % 20 == 0:  # Show progress every 20%
                    print(f"   📊 Progress: {progress}%")

        # Save to local file
        local_filename = file_info['name']
        with open(local_filename, 'wb') as f:
            f.write(downloaded_content.getvalue())

        file_size_mb = os.path.getsize(local_filename) / (1024 * 1024)
        print(f"✅ Downloaded: {local_filename} ({file_size_mb:.1f} MB)")
        return local_filename

    except Exception as e:
        print(f"❌ Download failed: {e}")
        return None

def handle_file_upload():
    """Handle local file upload with compression support."""
    print("📁 Upload your genome FASTA file:")
    print("   📋 Supported formats:")
    print("      • FASTA: .fasta, .fa, .fas, .fna")
    print("      • Compressed: .gz, .zip, .bz2")
    print("   📏 Requirements:")
    print("      • Minimum sequence length: 25 kbp per record")
    print("      • Maximum file size: 1 GB")
    print("      • Valid FASTA format with '>' headers")
    print()

    uploaded = files.upload()

    if not uploaded:
        print("❌ No file uploaded")
        return None

    uploaded_filename = list(uploaded.keys())[0]
    print(f"📋 Processing uploaded file: {uploaded_filename}")

    return process_fasta_file(uploaded_filename)

def process_fasta_file(filename):
    """Process FASTA file with decompression if needed."""
    global input_fasta_path, input_species_name, genome_stats

    # Determine file type and extract if needed
    temp_fasta_path = None

    try:
        if filename.endswith('.gz'):
            print("🗜️  Decompressing .gz file...")
            with gzip.open(filename, 'rt') as f:
                content = f.read()
            temp_fasta_path = filename.replace('.gz', '')
            with open(temp_fasta_path, 'w') as f:
                f.write(content)

        elif filename.endswith('.zip'):
            print("🗜️  Extracting .zip file...")
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('.')
                # Find FASTA file in extracted files
                for file in zip_ref.namelist():
                    if file.lower().endswith(('.fasta', '.fa', '.fas', '.fna')):
                        temp_fasta_path = file
                        break

        elif filename.endswith('.bz2'):
            print("🗜️  Decompressing .bz2 file...")
            with bz2.open(filename, 'rt') as f:
                content = f.read()
            temp_fasta_path = filename.replace('.bz2', '')
            with open(temp_fasta_path, 'w') as f:
                f.write(content)

        else:
            # Assume uncompressed FASTA
            temp_fasta_path = filename

        if not temp_fasta_path or not os.path.exists(temp_fasta_path):
            print("❌ Could not find or extract FASTA file")
            return None

        # Validate FASTA file
        with open(temp_fasta_path, 'r') as f:
            first_line = f.readline().strip()
            if not first_line.startswith('>'):
                print("❌ File does not appear to be in FASTA format")
                print("   First line should start with '>' character")
                return None

        # Calculate comprehensive genome statistics
        genome_stats = calculate_genome_statistics(temp_fasta_path)

        # Display statistics
        display_genome_statistics(genome_stats)

        # Check minimum sequence length requirement
        if genome_stats['shortest_sequence'] < 25000:
            print(f"\n⚠️  Warning: Shortest sequence ({genome_stats['shortest_sequence']:,} bp) is below recommended minimum (25,000 bp)")
            print("   This may affect annotation quality for short sequences")

        # Extract species name from filename - create shorter, cleaner names
        base_name = os.path.splitext(os.path.basename(temp_fasta_path))[0]

        # Create a much shorter species name
        if 'GCA_' in base_name or 'GCF_' in base_name:
            # For NCBI accessions, just use the first part
            species_name = base_name.split('_')[0] + '_' + base_name.split('_')[1]
        else:
            # For other names, take first 2 meaningful parts
            parts = re.sub(r'[_\-\.]+', ' ', base_name).split()[:2]
            species_name = '_'.join(parts)

        # Clean up further
        species_name = re.sub(r'[^A-Za-z0-9_]', '', species_name)

        # Ensure it's not empty
        if not species_name:
            species_name = "unknown_species"

        input_fasta_path = temp_fasta_path
        input_species_name = species_name

        print(f"\n🏷️  Detected species name: {input_species_name}")
        return temp_fasta_path

    except Exception as e:
        print(f"❌ Error processing FASTA file: {e}")
        return None

def upload_to_drive(file_path, job_name, file_type="result"):
    """Upload file to Google Drive in the output folder."""
    global drive_service, helixer_folder_id

    if not drive_service or not helixer_folder_id:
        print("❌ Google Drive not properly configured")
        return None

    try:
        filename = os.path.basename(file_path)
        print(f"☁️  Uploading {filename} to Google Drive...")

        # Create job subfolder
        job_folder_name = f"{job_name}_{file_type}"
        job_folder_metadata = {
            'name': job_folder_name,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': [helixer_folder_id]
        }
        job_folder = drive_service.files().create(body=job_folder_metadata).execute()
        job_folder_id = job_folder['id']

        # Upload file
        file_metadata = {
            'name': filename,
            'parents': [job_folder_id]
        }
        media = MediaFileUpload(file_path, resumable=True)
        uploaded_file = drive_service.files().create(
            body=file_metadata,
            media_body=media
        ).execute()

        file_url = f"https://drive.google.com/file/d/{uploaded_file['id']}/view"
        print(f"✅ Upload complete: {file_url}")
        return file_url

    except Exception as e:
        print(f"❌ Upload failed: {e}")
        return None

# Display instructions
print("📋 Google Drive File Path Instructions:")
print("═" * 50)
print("If using 'Google Drive Path' method:")
print()
print("📁 Path Format Examples:")
print("   • Root folder file: filename.fasta")
print("   • Subfolder file: MyFolder/genome.fasta.gz")
print("   • Nested folders: Data/Genomes/Species/assembly.fa")
print()
print("✅ Supported File Formats:")
print("   • FASTA: .fasta, .fa, .fas, .fna")
print("   • Compressed: .gz, .zip, .bz2")
print()
print("⚠️  Important Notes:")
print("   • Use forward slashes (/) for folder separation")
print("   • File names are case-sensitive")
print("   • Ensure file is not in Trash")
print("   • File must be accessible by your Google account")
print()

# Execute setup
print("🚀 Starting Google Drive Setup...")
print("═" * 40)

if not setup_google_drive():
    print("\n❌ Google Drive setup failed")
    print("⚠️  Cannot proceed without Google Drive access")
else:
    print(f"\n✅ Google Drive setup completed!")
    print(f"📁 Output folder: {output_folder_name}")

    # Handle input file based on method
    if input_method == "Upload File":
        print(f"\n📤 File Upload Method Selected")
        result = handle_file_upload()

    elif input_method == "Google Drive Path":
        print(f"\n☁️  Google Drive Path Method Selected")

        if not gdrive_file_path.strip():
            print("❌ No Google Drive path specified")
            print("   Please enter the path to your genome file")
            result = None
        else:
            print(f"🔍 Looking for file: {gdrive_file_path}")
            file_info = find_file_by_path(gdrive_file_path)

            if file_info:
                # Download file from Google Drive
                local_file = download_from_gdrive(file_info)
                if local_file:
                    result = process_fasta_file(local_file)
                else:
                    result = None
            else:
                result = None

    # Final status
    if result:
        print(f"\n🎉 Input file ready for processing!")
        print(f"📁 Local path: {input_fasta_path}")
        print(f"🏷️  Species: {input_species_name}")
        print(f"☁️  Output folder: {output_folder_name}")
    else:
        print(f"\n❌ File setup failed")
        print("⚠️  Please check your input method and try again")

📋 Google Drive File Path Instructions:
══════════════════════════════════════════════════
If using 'Google Drive Path' method:

📁 Path Format Examples:
   • Root folder file: filename.fasta
   • Subfolder file: MyFolder/genome.fasta.gz
   • Nested folders: Data/Genomes/Species/assembly.fa

✅ Supported File Formats:
   • FASTA: .fasta, .fa, .fas, .fna
   • Compressed: .gz, .zip, .bz2

⚠️  Important Notes:
   • Use forward slashes (/) for folder separation
   • File names are case-sensitive
   • Ensure file is not in Trash
   • File must be accessible by your Google account

🚀 Starting Google Drive Setup...
════════════════════════════════════════
🔐 Authenticating with Google Drive...
✅ Google Drive authentication successful
📁 Setting up output folder: Helixer_Output
✅ Found existing folder: Helixer_Output

✅ Google Drive setup completed!
📁 Output folder: Helixer_Output

☁️  Google Drive Path Method Selected
🔍 Looking for file: /Fusarium/GCA_001702875.1_Fol004_genomic.fasta.gz
🔍 Searchin

In [ ]:
# CELL 4: Job Configuration and Parameters
#@title Configure Helixer Parameters
#@markdown Set job name, lineage model, and prediction parameters. Parameters are intelligently recommended based on your genome assembly quality and lineage.

import re
from datetime import datetime

# Job configuration
job_name = "helixer_annotation" #@param {type:"string"}
#@markdown Job name for organizing output files

species_name_override = "" #@param {type:"string"}
#@markdown Override detected species name (leave empty to use detected name)

gff_feature_label = "Helixer" #@param {type:"string"}
#@markdown Label for GFF feature naming (appears in the source column)

# Model selection
lineage = "fungi" #@param ["land_plant", "vertebrate", "invertebrate", "fungi"]
#@markdown Choose the lineage model appropriate for your species

# Parameter automation mode
parameter_mode = "Auto-Optimized" #@param ["Auto-Optimized", "Conservative", "Manual"]
#@markdown **Auto-Optimized**: Best settings for your assembly quality and lineage (recommended). **Conservative**: Safe settings for any assembly. **Manual**: Full control over all parameters.

# Core prediction parameters (Manual mode only)
peak_threshold = 0.6 #@param {type:"slider", min:0.1, max:1.0, step:0.05}
#@markdown **Peak threshold**: Minimum genic score to accept a gene candidate. Lower values (0.6) increase sensitivity for effector annotation. Higher values (0.8-0.95) increase precision.

edge_threshold = 0.1 #@param {type:"slider", min:0.05, max:0.5, step:0.05}
#@markdown **Edge threshold**: Genic score threshold for defining start/end boundaries of candidate regions within the sliding window.

min_coding_length = 60 #@param {type:"integer"}
#@markdown **Minimum coding length**: Filter out genes with total coding sequence shorter than this value (bp). Recommended: 60 bp for effectors, 150+ bp for standard annotation.

window_size = 100 #@param {type:"integer"}
#@markdown **Window size**: Width of sliding window assessed for intergenic vs genic content. Default 100 bp works well for most genomes.

# Advanced parameters (Manual mode only)
subsequence_length = "auto" #@param ["auto", "21384", "64152", "106920", "213840", "320760", "custom"]
#@markdown **Subsequence length**: How much genome the neural network sees at once. Should be longer than typical genes. Auto-selected based on lineage and assembly quality.

custom_subsequence_length = 64152 #@param {type:"integer"}
#@markdown **Custom subsequence length**: Used only if 'custom' selected above. Must be divisible by 9.

batch_size = "auto" #@param ["auto", "8", "16", "32", "64", "128"]
#@markdown **Batch size**: Number of subsequences processed simultaneously. Larger values are faster but require more GPU memory. Auto-selected for optimal performance.

use_overlap = True #@param {type:"boolean"}
#@markdown **Use overlap**: Create overlapping sliding-window predictions for better quality at subsequence boundaries. Increases processing time but improves accuracy.

predict_phase = True #@param {type:"boolean"}
#@markdown **Predict phase**: Also predict CDS reading frames/phases. Recommended for complete gene models and downstream analysis.

verbose_output = True #@param {type:"boolean"}
#@markdown **Verbose output**: Enable detailed progress information during processing.

def get_intelligent_parameters(lineage, assembly_stats, mode="Auto-Optimized"):
    """Get intelligent parameter recommendations based on lineage and assembly quality."""

    if not assembly_stats:
        print("⚠️  No assembly statistics available, using conservative defaults")
        return get_conservative_parameters(lineage)

    n50 = assembly_stats['n50']
    quality_class = assembly_stats['quality_class']

    # Define lineage-specific parameter matrices
    # Format: [subsequence_length, batch_size, overlap_offset, overlap_core_length]

    lineage_configs = {
        "fungi": {
            "conservative":   [21384,  64,  10692,  16038],   # GitHub defaults
            "standard":       [64152,  128, 32076,  48114],   # 3x faster, good assemblies
            "aggressive":     [106920, 128, 53460,  80190],   # 5x faster, excellent assemblies
        },
        "land_plant": {
            "conservative":   [64152,  64,  32076,  48114],   # GitHub defaults
            "standard":       [106920, 64,  53460,  80190],   # GitHub "try up to"
            "aggressive":     [213840, 32,  106920, 160380],  # Vertebrate-level for excellent assemblies
        },
        "vertebrate": {
            "conservative":   [106920, 32,  53460,  80190],   # For fragmented assemblies
            "standard":       [213840, 32,  106920, 160380],  # GitHub defaults
            "aggressive":     [320760, 16,  160380, 240570],  # For chromosome-level assemblies
        },
        "invertebrate": {
            "conservative":   [106920, 32,  53460,  80190],   # For fragmented assemblies
            "standard":       [213840, 32,  106920, 160380],  # GitHub defaults
            "aggressive":     [320760, 16,  160380, 240570],  # For chromosome-level assemblies
        }
    }

    config = lineage_configs.get(lineage, lineage_configs["land_plant"])

    # Select parameter set based on assembly quality and mode
    if mode == "Conservative":
        params = config["conservative"]
        param_desc = "Conservative (safe for any assembly)"
    elif mode == "Auto-Optimized":
        # Intelligent selection based on N50
        if quality_class == "Excellent":  # N50 > 1 Mb
            params = config["aggressive"]
            param_desc = f"Aggressive (optimized for {quality_class.lower()} assembly)"
        elif quality_class == "Good":     # N50 100kb-1Mb
            params = config["standard"]
            param_desc = f"Standard (optimized for {quality_class.lower()} assembly)"
        else:                            # N50 < 100kb
            params = config["conservative"]
            param_desc = f"Conservative (safe for {quality_class.lower()} assembly)"
    else:  # Manual
        return None, None

    # Validate that subsequence length doesn't exceed N50
    subsequence_length, batch_size, overlap_offset, overlap_core_length = params

    # Safety check: reduce if subsequence length > 50% of N50
    max_safe_length = int(n50 * 0.5)
    if subsequence_length > max_safe_length:
        print(f"⚠️  Reducing subsequence length from {subsequence_length:,} to {max_safe_length:,} bp (50% of N50)")
        subsequence_length = max_safe_length
        # Recalculate dependent parameters
        overlap_offset = subsequence_length // 2
        overlap_core_length = int(subsequence_length * 0.75)
        param_desc += " (N50-adjusted)"

    return {
        'subsequence_length': subsequence_length,
        'batch_size': batch_size,
        'overlap_offset': overlap_offset,
        'overlap_core_length': overlap_core_length,
        'description': param_desc
    }, param_desc

def get_conservative_parameters(lineage):
    """Get conservative parameters that work for any assembly quality."""
    conservative_params = {
        "fungi": [21384, 64, 10692, 16038],
        "land_plant": [64152, 64, 32076, 48114],
        "vertebrate": [106920, 32, 53460, 80190],
        "invertebrate": [106920, 32, 53460, 80190]
    }

    params = conservative_params.get(lineage, conservative_params["land_plant"])
    subsequence_length, batch_size, overlap_offset, overlap_core_length = params

    return {
        'subsequence_length': subsequence_length,
        'batch_size': batch_size,
        'overlap_offset': overlap_offset,
        'overlap_core_length': overlap_core_length,
        'description': "Conservative (safe for any assembly)"
    }, "Conservative (safe for any assembly)"

def validate_and_prepare_config():
    """Validate parameters and prepare configuration."""
    global job_name, species_name_override, input_species_name, genome_stats

    # Validate required inputs
    if not input_fasta_path:
        print("❌ No input file uploaded. Please run the file upload cell first.")
        return False

    # Clean job name
    job_name_clean = re.sub(r'[^\w\-_]', '_', job_name.strip())
    if not job_name_clean:
        job_name_clean = f"helixer_job_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

    # Set species name
    final_species = species_name_override.strip() if species_name_override.strip() else input_species_name

    # Get intelligent parameter recommendations
    if parameter_mode in ["Auto-Optimized", "Conservative"]:
        optimal_params, param_desc = get_intelligent_parameters(lineage, genome_stats, parameter_mode)

        if optimal_params:
            print(f"\n🎯 Parameter Optimization:")
            print(f"   📊 Assembly quality: {genome_stats['quality_class']} (N50: {genome_stats['n50']/1000:.1f} kb)")
            print(f"   🌿 Lineage: {lineage}")
            print(f"   ⚙️  Selected: {param_desc}")
            print(f"   📏 Subsequence length: {optimal_params['subsequence_length']:,} bp")
            print(f"   🚀 Expected speed improvement: {optimal_params['subsequence_length'] // 21384:.1f}x faster than minimum settings")

            final_subsequence_length = optimal_params['subsequence_length']
            optimal_batch_size = optimal_params['batch_size']
            overlap_offset = optimal_params['overlap_offset']
            overlap_core_length = optimal_params['overlap_core_length']

            # Use optimized thresholds for effector annotation
            final_peak_threshold = 0.6  # Optimized for sensitivity
            final_edge_threshold = 0.1
        else:
            print("❌ Could not determine optimal parameters")
            return False
    else:
        # Manual mode - use user inputs
        print(f"\n⚙️  Manual parameter mode selected")

        if subsequence_length == "auto":
            # Use conservative auto settings even in manual mode
            conservative_params, _ = get_conservative_parameters(lineage)
            final_subsequence_length = conservative_params['subsequence_length']
        elif subsequence_length == "custom":
            final_subsequence_length = custom_subsequence_length
        else:
            final_subsequence_length = int(subsequence_length)

        optimal_batch_size = int(batch_size) if batch_size != "auto" else 32

        # Calculate overlap parameters
        if use_overlap:
            overlap_offset = final_subsequence_length // 2
            overlap_core_length = int(final_subsequence_length * 0.75)
        else:
            overlap_offset = None
            overlap_core_length = None

        final_peak_threshold = peak_threshold
        final_edge_threshold = edge_threshold

        # Validate against N50 in manual mode
        if genome_stats and final_subsequence_length > genome_stats['n50'] * 0.5:
            print(f"⚠️  Warning: Subsequence length ({final_subsequence_length:,} bp) is large relative to N50 ({genome_stats['n50']/1000:.1f} kb)")
            print("   This may reduce prediction quality. Consider using Auto-Optimized mode.")

    # Create configuration dictionary
    config = {
        'job_name': job_name_clean,
        'species_name': final_species,
        'gff_feature_label': gff_feature_label,
        'lineage': lineage,
        'parameter_mode': parameter_mode,
        'peak_threshold': final_peak_threshold,
        'edge_threshold': final_edge_threshold,
        'min_coding_length': min_coding_length,
        'window_size': window_size,
        'subsequence_length': final_subsequence_length,
        'batch_size': optimal_batch_size,
        'use_overlap': use_overlap,
        'overlap_offset': overlap_offset,
        'overlap_core_length': overlap_core_length,
        'predict_phase': predict_phase,
        'verbose_output': verbose_output,
        'input_fasta': input_fasta_path,
        'genome_stats': genome_stats
    }

    # Display configuration summary
    print("\n✅ Configuration validated successfully!")
    print(f"\n📋 Job Configuration Summary:")
    print(f"   🏷️  Job name: {config['job_name']}")
    print(f"   🧬 Species: {config['species_name']}")
    print(f"   🌿 Lineage model: {config['lineage']}")
    print(f"   ⚙️  Parameter mode: {config['parameter_mode']}")
    print(f"   📊 Peak threshold: {config['peak_threshold']} (effector-optimized)")
    print(f"   📏 Subsequence length: {config['subsequence_length']:,} bp")
    print(f"   🔄 Use overlap: {config['use_overlap']}")
    print(f"   🧩 Batch size: {config['batch_size']}")
    print(f"   📖 Predict phases: {config['predict_phase']}")

    return config

# Execute configuration validation
prediction_config = validate_and_prepare_config()

if prediction_config:
    print("\n🎉 Configuration ready for prediction!")
    if prediction_config['peak_threshold'] <= 0.6:
        print("🎯 Using optimized peak threshold for effector annotation sensitivity")
    if prediction_config['parameter_mode'] == "Auto-Optimized":
        print("⚡ Using assembly-quality optimized parameters for best performance")
else:
    print("\n❌ Configuration validation failed")
    print("⚠️  Please check parameters and input file")


🎯 Parameter Optimization:
   📊 Assembly quality: Good (N50: 152.3 kb)
   🌿 Lineage: fungi
   ⚙️  Selected: Standard (optimized for good assembly)
   📏 Subsequence length: 64,152 bp
   🚀 Expected speed improvement: 3.0x faster than minimum settings

✅ Configuration validated successfully!

📋 Job Configuration Summary:
   🏷️  Job name: helixer_annotation
   🧬 Species: GCA_0017028751
   🌿 Lineage model: fungi
   ⚙️  Parameter mode: Auto-Optimized
   📊 Peak threshold: 0.6 (effector-optimized)
   📏 Subsequence length: 64,152 bp
   🔄 Use overlap: True
   🧩 Batch size: 128
   📖 Predict phases: True

🎉 Configuration ready for prediction!
🎯 Using optimized peak threshold for effector annotation sensitivity
⚡ Using assembly-quality optimized parameters for best performance


In [ ]:
# CELL 5: Download Helixer Models
#@title Download Lineage Models
#@markdown Download the appropriate trained model for your selected lineage.

import urllib.request
import hashlib

def download_helixer_models(target_lineage):
    """Download Helixer models for the specified lineage."""

    print(f"📥 Downloading Helixer models for lineage: {target_lineage}")

    # Create models directory
    models_dir = "/content/helixer_models"
    lineage_dir = os.path.join(models_dir, target_lineage)
    os.makedirs(lineage_dir, exist_ok=True)

    # Model URLs and info
    model_info = {
        "fungi": {
            "filename": "fungi_v0.3_a_0100.h5",
            "url": "https://zenodo.org/records/10836346/files/fungi_v0.3_a_0100.h5"
        },
        "land_plant": {
            "filename": "land_plant_v0.3_a_0080.h5",
            "url": "https://zenodo.org/records/10836346/files/land_plant_v0.3_a_0080.h5"
        },
        "vertebrate": {
            "filename": "vertebrate_v0.3_m_0080.h5",
            "url": "https://zenodo.org/records/10836346/files/vertebrate_v0.3_m_0080.h5"
        },
        "invertebrate": {
            "filename": "invertebrate_v0.3_m_0100.h5",
            "url": "https://zenodo.org/records/10836346/files/invertebrate_v0.3_m_0100.h5"
        }
    }

    if target_lineage not in model_info:
        print(f"❌ Unknown lineage: {target_lineage}")
        return None

    model_data = model_info[target_lineage]
    model_path = os.path.join(lineage_dir, model_data["filename"])

    # Check if model already exists
    if os.path.exists(model_path):
        print(f"✅ Model already exists: {model_path}")
        return model_path

    try:
        print(f"🌐 Downloading {model_data['filename']}...")
        print("⏳ This may take several minutes depending on connection speed...")

        def progress_hook(block_num, block_size, total_size):
            if total_size > 0:
                percent = min(100, (block_num * block_size * 100) // total_size)
                if block_num % 100 == 0:  # Update every 100 blocks
                    print(f"   📊 Progress: {percent}% ({(block_num * block_size) // (1024*1024)} MB)")

        urllib.request.urlretrieve(
            model_data["url"],
            model_path,
            reporthook=progress_hook
        )

        # Verify download
        if os.path.exists(model_path):
            file_size = os.path.getsize(model_path)
            print(f"✅ Download completed: {file_size // (1024*1024)} MB")
            return model_path
        else:
            print("❌ Download failed: file not found")
            return None

    except Exception as e:
        print(f"❌ Download error: {e}")
        return None

# Download model for configured lineage
if prediction_config:
    model_path = download_helixer_models(prediction_config['lineage'])
    if model_path:
        prediction_config['model_path'] = model_path
        print(f"\n🎉 Model ready: {os.path.basename(model_path)}")
    else:
        print("\n❌ Model download failed")
else:
    print("❌ Please configure parameters first")

📥 Downloading Helixer models for lineage: fungi
🌐 Downloading fungi_v0.3_a_0100.h5...
⏳ This may take several minutes depending on connection speed...
   📊 Progress: 0% (0 MB)
   📊 Progress: 3% (0 MB)
   📊 Progress: 6% (1 MB)
   📊 Progress: 9% (2 MB)
   📊 Progress: 13% (3 MB)
   📊 Progress: 16% (3 MB)
   📊 Progress: 19% (4 MB)
   📊 Progress: 22% (5 MB)
   📊 Progress: 26% (6 MB)
   📊 Progress: 29% (7 MB)
   📊 Progress: 32% (7 MB)
   📊 Progress: 35% (8 MB)
   📊 Progress: 39% (9 MB)
   📊 Progress: 42% (10 MB)
   📊 Progress: 45% (10 MB)
   📊 Progress: 48% (11 MB)
   📊 Progress: 52% (12 MB)
   📊 Progress: 55% (13 MB)
   📊 Progress: 58% (14 MB)
   📊 Progress: 62% (14 MB)
   📊 Progress: 65% (15 MB)
   📊 Progress: 68% (16 MB)
   📊 Progress: 71% (17 MB)
   📊 Progress: 75% (17 MB)
   📊 Progress: 78% (18 MB)
   📊 Progress: 81% (19 MB)
   📊 Progress: 84% (20 MB)
   📊 Progress: 88% (21 MB)
   📊 Progress: 91% (21 MB)
   📊 Progress: 94% (22 MB)
   📊 Progress: 97% (23 MB)
✅ Download completed: 23 MB



In [ ]:
# CELL 6: Run Helixer Prediction Pipeline with Results
#@title Execute Helixer Gene Annotation and Download Results
#@markdown Run the complete Helixer pipeline and immediately download/upload results to prevent data loss.

import subprocess
import time
import glob
from datetime import datetime
import zipfile
from google.colab import files

def run_helixer_pipeline_with_results(config):
    """Execute complete Helixer pipeline and handle results immediately."""

    if not config or 'model_path' not in config:
        print("❌ Configuration or model not ready")
        return False

    print("🚀 Starting Helixer Gene Annotation Pipeline")
    print(f"⏰ Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 60)

    # FIX: Get absolute path BEFORE changing directories
    input_fasta = os.path.abspath(config['input_fasta'])
    print(f"🔍 Input FASTA absolute path: {input_fasta}")

    # Verify file exists
    if not os.path.exists(input_fasta):
        print(f"❌ Input FASTA file not found: {input_fasta}")
        return False

    # Create working directory
    work_dir = f"/content/{config['job_name']}_helixer_work"
    os.makedirs(work_dir, exist_ok=True)
    os.chdir(work_dir)

    # File paths
    h5_file = f"{config['job_name']}.h5"
    predictions_file = "predictions.h5"
    final_gff = f"{config['job_name']}_helixer_annotation.gff3"

    pipeline_start = time.time()

    try:
        # Step 1: Convert FASTA to H5
        print("\n🔄 Step 1/5: Converting FASTA to numerical matrices (fasta2h5)")
        print(f"   📁 Input: {os.path.basename(input_fasta)}")
        print(f"   📁 Output: {h5_file}")
        print(f"   📏 Subsequence length: {config['subsequence_length']:,} bp")

        fasta2h5_cmd = [
            "python3", "/content/Helixer/fasta2h5.py",
            "--species", config['species_name'],
            "--h5-output-path", h5_file,
            "--fasta-path", input_fasta,
            "--subsequence-length", str(config['subsequence_length'])
        ]

        if config['verbose_output']:
            print(f"   💻 Command: {' '.join(fasta2h5_cmd)}")

        step1_start = time.time()
        result1 = subprocess.run(fasta2h5_cmd, capture_output=True, text=True)
        step1_time = time.time() - step1_start

        if result1.returncode != 0:
            print(f"❌ fasta2h5 failed: {result1.stderr}")
            return False

        print(f"✅ Step 1 completed in {step1_time:.1f}s")

        # Step 2: Neural Network Prediction
        print("\n🧠 Step 2/5: Neural network prediction (HybridModel)")
        print(f"   🤖 Model: {os.path.basename(config['model_path'])}")
        print(f"   📊 Batch size: {config['batch_size']}")
        print(f"   ⚙️  Parameter mode: {config['parameter_mode']}")

        hybrid_cmd = [
            "python3", "/content/Helixer/helixer/prediction/HybridModel.py",
            "--load-model-path", config['model_path'],
            "--test-data", h5_file,
            "--val-test-batch-size", str(config['batch_size'])
        ]

        if config['use_overlap']:
            hybrid_cmd.append("--overlap")

        if config['predict_phase']:
            hybrid_cmd.append("--predict-phase")

        if config['verbose_output']:
            hybrid_cmd.append("--verbose")
            print(f"   💻 Command: {' '.join(hybrid_cmd)}")

        step2_start = time.time()
        result2 = subprocess.run(hybrid_cmd, capture_output=True, text=True)
        step2_time = time.time() - step2_start

        if result2.returncode != 0:
            print(f"❌ HybridModel failed: {result2.stderr}")
            return False

        print(f"✅ Step 2 completed in {step2_time:.1f}s")

        # Step 3: Post-processing to GFF3
        print("\n📝 Step 3/5: Post-processing to GFF3 (helixer_post_bin)")
        print(f"   🎯 Peak threshold: {config['peak_threshold']} (effector-optimized)")
        print(f"   📏 Min coding length: {config['min_coding_length']} bp")

        post_cmd = [
            "helixer_post_bin",
            h5_file,
            predictions_file,
            str(config['window_size']),
            str(config['edge_threshold']),
            str(config['peak_threshold']),
            str(config['min_coding_length']),
            final_gff
        ]

        if config['verbose_output']:
            print(f"   💻 Command: {' '.join(post_cmd)}")

        step3_start = time.time()
        result3 = subprocess.run(post_cmd, capture_output=True, text=True)
        step3_time = time.time() - step3_start

        if result3.returncode != 0:
            print(f"❌ Post-processing failed: {result3.stderr}")
            return False

        print(f"✅ Step 3 completed in {step3_time:.1f}s")

        # Validate output
        if not os.path.exists(final_gff):
            print("❌ Final GFF3 file not generated")
            return False

        # Step 4: Extract sequences with gffread and create Geneious-compatible GFF
        print("\n📋 Step 4/5: Extracting sequences and creating Geneious-compatible GFF")

        # Extract proteins
        proteins_file = f"{config['job_name']}_proteins.fasta"
        gffread_cmd = ["gffread", "-y", proteins_file, "-g", input_fasta, final_gff]
        result_proteins = subprocess.run(gffread_cmd, capture_output=True, text=True)
        if result_proteins.returncode == 0:
            print(f"   ✅ Proteins extracted: {proteins_file}")
        else:
            print(f"   ⚠️  Protein extraction failed: {result_proteins.stderr}")

        # Extract transcripts
        transcripts_file = f"{config['job_name']}_transcripts.fasta"
        gffread_cmd = ["gffread", "-w", transcripts_file, "-g", input_fasta, final_gff]
        result_transcripts = subprocess.run(gffread_cmd, capture_output=True, text=True)
        if result_transcripts.returncode == 0:
            print(f"   ✅ Transcripts extracted: {transcripts_file}")
        else:
            print(f"   ⚠️  Transcript extraction failed: {result_transcripts.stderr}")

        # Extract CDS
        cds_file = f"{config['job_name']}_cds.fasta"
        gffread_cmd = ["gffread", "-x", cds_file, "-g", input_fasta, final_gff]
        result_cds = subprocess.run(gffread_cmd, capture_output=True, text=True)
        if result_cds.returncode == 0:
            print(f"   ✅ CDS extracted: {cds_file}")
        else:
            print(f"   ⚠️  CDS extraction failed: {result_cds.stderr}")

        # Extract gene sequences
        genes_file = f"{config['job_name']}_genes.fasta"
        gffread_cmd = ["gffread", "-G", genes_file, "-g", input_fasta, final_gff]
        result_genes = subprocess.run(gffread_cmd, capture_output=True, text=True)
        if result_genes.returncode == 0:
            print(f"   ✅ Genes extracted: {genes_file}")
        else:
            print(f"   ⚠️  Gene extraction failed: {result_genes.stderr}")

        # Create Geneious-compatible GFF
        print("\n🔧 Step 5/5: Creating Geneious-compatible GFF")
        geneious_gff = f"{config['job_name']}_geneious.gff3"

        with open(final_gff, 'r') as infile, open(geneious_gff, 'w') as outfile:
            for line in infile:
                if line.startswith('#'):
                    outfile.write(line)
                    continue

                if line.strip():
                    fields = line.strip().split('\t')
                    if len(fields) >= 9:
                        # Clean up the attributes for Geneious compatibility
                        attributes = fields[8]

                        # Remove feature-specific numbering from IDs
                        attributes = re.sub(r'\.exon\.\d+', '', attributes)
                        attributes = re.sub(r'\.CDS\.\d+', '', attributes)
                        attributes = re.sub(r'\.five_prime_UTR\.\d+', '', attributes)
                        attributes = re.sub(r'\.three_prime_UTR\.\d+', '', attributes)

                        # Remove frame info for CDS to make identical IDs
                        if fields[2] == 'CDS':
                            fields[7] = '.'  # Remove phase/frame

                        fields[8] = attributes
                        outfile.write('\t'.join(fields) + '\n')

        print(f"   ✅ Geneious-compatible GFF created: {geneious_gff}")

        # Parse GFF3 for detailed statistics
        print("\n📊 Analyzing annotation results...")
        feature_counts = {}
        total_cds_length = 0
        gene_lengths = []
        exon_counts = {}

        with open(final_gff, 'r') as f:
            current_gene_length = 0
            current_gene_exons = 0

            for line in f:
                if line.startswith('#') or not line.strip():
                    continue

                fields = line.strip().split('\t')
                if len(fields) >= 9:
                    feature_type = fields[2]
                    start, end = int(fields[3]), int(fields[4])
                    length = end - start + 1

                    feature_counts[feature_type] = feature_counts.get(feature_type, 0) + 1

                    if feature_type == 'gene':
                        if current_gene_length > 0:
                            gene_lengths.append(current_gene_length)
                            exon_counts[current_gene_exons] = exon_counts.get(current_gene_exons, 0) + 1
                        current_gene_length = length
                        current_gene_exons = 0
                    elif feature_type == 'exon':
                        current_gene_exons += 1
                    elif feature_type == 'CDS':
                        total_cds_length += length

            # Handle last gene
            if current_gene_length > 0:
                gene_lengths.append(current_gene_length)
                exon_counts[current_gene_exons] = exon_counts.get(current_gene_exons, 0) + 1

        pipeline_time = time.time() - pipeline_start

        # Create comprehensive results package
        print("\n📦 Creating comprehensive results package...")
        results_zip = f"/content/{config['job_name']}_helixer_results.zip"

        with zipfile.ZipFile(results_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Add GFF3 files
            zipf.write(final_gff, os.path.basename(final_gff))
            if os.path.exists(geneious_gff):
                zipf.write(geneious_gff, os.path.basename(geneious_gff))

            # Add extracted sequences
            if os.path.exists(proteins_file):
                zipf.write(proteins_file, os.path.basename(proteins_file))
            if os.path.exists(transcripts_file):
                zipf.write(transcripts_file, os.path.basename(transcripts_file))
            if os.path.exists(cds_file):
                zipf.write(cds_file, os.path.basename(cds_file))
            if os.path.exists(genes_file):
                zipf.write(genes_file, os.path.basename(genes_file))

            # Create detailed summary file
            summary_file = f"{config['job_name']}_summary.txt"
            with open(summary_file, 'w') as f:
                f.write("Helixer Gene Annotation Results\n")
                f.write("=" * 40 + "\n\n")

                # Job information
                f.write("JOB INFORMATION:\n")
                f.write(f"Job name: {config['job_name']}\n")
                f.write(f"Species: {config['species_name']}\n")
                f.write(f"Lineage model: {config['lineage']}\n")
                f.write(f"Parameter mode: {config['parameter_mode']}\n")
                f.write(f"Peak threshold: {config['peak_threshold']}\n")
                f.write(f"Edge threshold: {config['edge_threshold']}\n")
                f.write(f"Min coding length: {config['min_coding_length']} bp\n")
                f.write(f"Subsequence length: {config['subsequence_length']:,} bp\n")
                f.write(f"Processing time: {pipeline_time:.1f}s ({pipeline_time/60:.1f} minutes)\n")
                f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

                # Assembly statistics
                if config['genome_stats']:
                    stats = config['genome_stats']
                    f.write("ASSEMBLY STATISTICS:\n")
                    f.write(f"Assembly quality: {stats['quality_class']} ({stats['quality_desc']})\n")
                    f.write(f"Genome size: {stats['total_length'] / 1_000_000:.1f} Mb\n")
                    f.write(f"Number of sequences: {stats['sequence_count']:,}\n")
                    f.write(f"Sequence N50: {stats['n50'] / 1000:.1f} kb\n")
                    f.write(f"Sequence N90: {stats['n90'] / 1000:.1f} kb\n")
                    f.write(f"GC content: {stats['gc_percent']:.1f}%\n\n")

                # Feature statistics
                f.write("ANNOTATION STATISTICS:\n")
                for feature, count in sorted(feature_counts.items()):
                    f.write(f"{feature}: {count:,}\n")
                f.write(f"\n")

                # Gene statistics
                if gene_lengths:
                    f.write("GENE LENGTH STATISTICS:\n")
                    f.write(f"Total genes: {len(gene_lengths):,}\n")
                    f.write(f"Average gene length: {sum(gene_lengths) / len(gene_lengths):,.0f} bp\n")
                    f.write(f"Shortest gene: {min(gene_lengths):,} bp\n")
                    f.write(f"Longest gene: {max(gene_lengths):,} bp\n")
                    f.write(f"Median gene length: {sorted(gene_lengths)[len(gene_lengths)//2]:,} bp\n")
                    f.write(f"Total coding length: {total_cds_length:,} bp\n\n")

                # Exon distribution
                if exon_counts:
                    f.write("EXON DISTRIBUTION:\n")
                    for exon_count in sorted(exon_counts.keys()):
                        if exon_count > 0:
                            f.write(f"Genes with {exon_count} exons: {exon_counts[exon_count]:,}\n")

                f.write(f"\n")
                f.write("FILES INCLUDED:\n")
                f.write(f"• {os.path.basename(final_gff)} - Standard Helixer GFF3 annotation\n")
                f.write(f"• {os.path.basename(geneious_gff)} - Geneious-compatible GFF3\n")
                f.write(f"• {os.path.basename(proteins_file)} - Protein sequences\n")
                f.write(f"• {os.path.basename(transcripts_file)} - Transcript sequences\n")
                f.write(f"• {os.path.basename(cds_file)} - CDS sequences\n")
                f.write(f"• {os.path.basename(genes_file)} - Gene sequences\n\n")

                f.write("NEXT STEPS:\n")
                f.write(f"• Validate with BUSCO: busco -i {os.path.basename(proteins_file)} -l {config['lineage']}_odb10 -m proteins\n")
                f.write("• Visualize in genome browsers (IGV, JBrowse, Geneious)\n")
                f.write("• Perform functional annotation (InterProScan, eggNOG-mapper)\n")
                f.write("• Compare with other gene predictors for consensus\n")

            zipf.write(summary_file, os.path.basename(summary_file))

            # Add configuration file
            config_file = f"{config['job_name']}_config.txt"
            with open(config_file, 'w') as f:
                f.write("Helixer Configuration Used\n")
                f.write("=" * 30 + "\n\n")
                for key, value in config.items():
                    if key not in ['input_fasta', 'genome_stats']:  # Skip file path and complex objects
                        f.write(f"{key}: {value}\n")

            zipf.write(config_file, os.path.basename(config_file))

        file_size_kb = os.path.getsize(results_zip) / 1024
        print(f"✅ Results package created: {os.path.basename(results_zip)} ({file_size_kb:.1f} KB)")

        # Display comprehensive results summary
        print("\n🎉 Helixer Pipeline Completed Successfully!")
        print("=" * 60)
        print(f"⏰ Total processing time: {pipeline_time:.1f}s ({pipeline_time/60:.1f} minutes)")
        if config['genome_stats']:
            stats = config['genome_stats']
            print(f"📊 Assembly: {stats['quality_class']} quality ({stats['n50']/1000:.1f} kb N50)")
        print(f"\n📊 Annotation Results:")
        print(f"   🧬 Predicted genes: {feature_counts.get('gene', 0):,}")
        print(f"   🔬 CDS features: {feature_counts.get('CDS', 0):,}")
        print(f"   🧪 Exon features: {feature_counts.get('exon', 0):,}")

        if gene_lengths:
            avg_gene_length = sum(gene_lengths) / len(gene_lengths)
            print(f"   📏 Average gene length: {avg_gene_length:,.0f} bp")
            print(f"   📐 Total coding length: {total_cds_length:,} bp")

        print(f"\n📁 Files generated:")
        print(f"   • {os.path.basename(final_gff)} (standard annotation)")
        print(f"   • {os.path.basename(geneious_gff)} (Geneious-compatible)")
        print(f"   • {os.path.basename(proteins_file)} (protein sequences)")
        print(f"   • {os.path.basename(transcripts_file)} (transcript sequences)")
        print(f"   • {os.path.basename(cds_file)} (CDS sequences)")
        print(f"   • {os.path.basename(genes_file)} (gene sequences)")
        print(f"   • {os.path.basename(results_zip)} (complete package)")

        # Immediate upload to Google Drive
        drive_url = None
        if drive_service and helixer_folder_id:
            print(f"\n☁️  Uploading to Google Drive...")
            drive_url = upload_to_drive(results_zip, config['job_name'], "annotation")
            if drive_url:
                print(f"✅ Google Drive upload successful!")
                print(f"🔗 Access link: {drive_url}")

        # Immediate local download
        print(f"\n⬇️  Initiating local download...")
        try:
            files.download(results_zip)
            print("✅ Local download initiated")
        except Exception as e:
            print(f"⚠️  Local download error: {e}")
            print("📁 File saved in Colab session - manually download if needed")

        # Cleanup intermediate files
        print(f"\n🧹 Cleaning up intermediate files...")
        cleanup_files = [h5_file, predictions_file, summary_file, config_file]
        for temp_file in cleanup_files:
            if os.path.exists(temp_file):
                os.remove(temp_file)
                print(f"   🗑️  Removed: {temp_file}")

        print(f"\n🏁 Annotation pipeline completed successfully!")
        print(f"📋 Results summary:")
        print(f"   • Standard GFF3 with {feature_counts.get('gene', 0):,} genes")
        print(f"   • Geneious-compatible GFF3 for visualization")
        print(f"   • Extracted protein, transcript, CDS, and gene sequences")
        print(f"   • Comprehensive results package with statistics")
        print(f"   • Google Drive backup: {output_folder_name}")
        print(f"   • Local download initiated")

        return True

    except Exception as e:
        print(f"\n❌ Pipeline error: {e}")
        return False

    finally:
        # Return to content directory
        os.chdir("/content")

# Execute the complete pipeline with immediate results handling
if prediction_config:
    success = run_helixer_pipeline_with_results(prediction_config)
    if success:
        print("\n🎊 Gene annotation completed successfully!")
        print("📋 Your results are now available in:")
        print("   • Local download (started automatically)")
        print(f"   • Google Drive: {output_folder_name} folder")
        print("\n💡 Next steps:")
        print("   • Import Geneious-compatible GFF into genome viewers")
        print("   • Use extracted sequences for functional annotation")
        print("   • Validate annotations with BUSCO")
        print("   • Compare with other gene predictors")
    else:
        print("\n❌ Pipeline failed - check error messages above")
        print("🔧 See troubleshooting guide below for common solutions")
else:
    print("❌ Please configure parameters and download models first")

🚀 Starting Helixer Gene Annotation Pipeline
⏰ Started at: 2025-06-25 16:28:04
🔍 Input FASTA absolute path: /content/GCA_001702875.1_Fol004_genomic.fasta

🔄 Step 1/5: Converting FASTA to numerical matrices (fasta2h5)
   📁 Input: GCA_001702875.1_Fol004_genomic.fasta
   📁 Output: helixer_annotation.h5
   📏 Subsequence length: 64,152 bp
   💻 Command: python3 /content/Helixer/fasta2h5.py --species GCA_0017028751 --h5-output-path helixer_annotation.h5 --fasta-path /content/GCA_001702875.1_Fol004_genomic.fasta --subsequence-length 64152
✅ Step 1 completed in 64.0s

🧠 Step 2/5: Neural network prediction (HybridModel)
   🤖 Model: fungi_v0.3_a_0100.h5
   📊 Batch size: 128
   ⚙️  Parameter mode: Auto-Optimized
   💻 Command: python3 /content/Helixer/helixer/prediction/HybridModel.py --load-model-path /content/helixer_models/fungi/fungi_v0.3_a_0100.h5 --test-data helixer_annotation.h5 --val-test-batch-size 128 --overlap --predict-phase --verbose
✅ Step 2 completed in 428.0s

📝 Step 3/5: Post-proces

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Local download initiated

🧹 Cleaning up intermediate files...
   🗑️  Removed: helixer_annotation.h5
   🗑️  Removed: predictions.h5
   🗑️  Removed: helixer_annotation_summary.txt
   🗑️  Removed: helixer_annotation_config.txt

🏁 Annotation pipeline completed successfully!
📋 Results summary:
   • Standard GFF3 with 20,545 genes
   • Geneious-compatible GFF3 for visualization
   • Extracted protein, transcript, CDS, and gene sequences
   • Comprehensive results package with statistics
   • Google Drive backup: Helixer_Output
   • Local download initiated

🎊 Gene annotation completed successfully!
📋 Your results are now available in:
   • Local download (started automatically)
   • Google Drive: Helixer_Output folder

💡 Next steps:
   • Import Geneious-compatible GFF into genome viewers
   • Use extracted sequences for functional annotation
   • Validate annotations with BUSCO
   • Compare with other gene predictors


# 🔧 Helixer Troubleshooting Guide & Parameter Reference

## Assembly Quality & Parameter Guidelines

### 🏆 **Excellent Assemblies (N50 > 1 Mb)**
**Characteristics**: Chromosome-level, highly contiguous  
**Optimized Settings**:
- **Fungi**: 106,920 bp subsequences (5x faster)
- **Plants**: 213,840 bp subsequences (3.3x faster)  
- **Vertebrates/Invertebrates**: 320,760 bp subsequences (1.5x faster)
- **Benefits**: Maximum speed, better gene context, improved accuracy

### 🥈 **Good Assemblies (N50 100kb-1Mb)**
**Characteristics**: Scaffold-level, good contiguity  
**Standard Settings**:
- **Fungi**: 64,152 bp subsequences (3x faster)
- **Plants**: 106,920 bp subsequences (1.7x faster)
- **Vertebrates/Invertebrates**: 213,840 bp subsequences (standard)
- **Benefits**: Balanced speed and safety

### 🥉 **Fair Assemblies (N50 10-100kb)**
**Characteristics**: Contig-level, moderate fragmentation  
**Conservative Settings**:
- **All lineages**: Use GitHub defaults or smaller
- **Safety**: Parameters guaranteed to work
- **Trade-off**: Slower but reliable

### ⚠️ **Poor Assemblies (N50 < 10kb)**
**Characteristics**: Highly fragmented  
**Minimal Settings**:
- Use smallest possible subsequence lengths
- Expect reduced annotation quality
- Consider improving assembly first

## Parameter Explanations (from GitHub)

### **Core Prediction Parameters**

**--peak-threshold** (0.1-1.0, default: 0.8)
> Threshold specifies the minimum peak genic score required to accept the candidate region; the candidate region is accepted if it contains at least one window with a genic score above this threshold

- **0.6**: Optimized for effector annotation (high sensitivity)
- **0.8**: Standard annotation (balanced)
- **0.9-0.95**: High precision annotation (low false positives)

**--edge-threshold** (0.05-0.5, default: 0.1)  
> Threshold specifies the genic score which defines the start/end boundaries of each candidate region within the sliding window

**--min-coding-length** (default: 60)
> Output is filtered to remove genes with a total coding length shorter than this value

- **60 bp**: For effector genes (minimum)
- **150+ bp**: For standard annotation quality

**--window-size** (default: 100)
> Width of the sliding window that is assessed for intergenic vs genic (UTR/Coding Sequence/Intron) content

### **Performance Parameters**

**--subsequence-length**
> How to slice the genomic sequence. Set moderately longer than length of typical genic loci. Must be evenly divisible by the timestep width of the used model, which is typically 9.

- **Critical**: Must be < 50% of your assembly N50
- **Larger values**: Faster processing, better gene context
- **GitHub defaults**: vertebrate: 213840, land_plant: 64152, fungi: 21384, invertebrate: 213840

**--batch-size** (default: 32)
> The batch size for the raw predictions in TensorFlow. Should be as large as possible on your GPU to save prediction time.

**--overlap** / --no-overlap**
> Switches off the overlapping after predictions are made. Overlap will improve prediction quality at subsequence ends by creating and overlapping sliding-window predictions.

- **Benefits**: Better accuracy at sequence boundaries
- **Cost**: ~2x processing time
- **Recommended**: True for final annotations

**--predict-phase**
> Add this to also predict phases for CDS (recommended); format: [None, 0, 1, 2]; 'None' is used for non-CDS regions, within CDS regions 0, 1, 2 correspond to phase

## Common Issues and Solutions

### 🚫 **GPU Out of Memory**
**Problem**: CUDA out of memory during prediction  
**Solutions**:
- Reduce batch size (32 → 16 → 8)
- Use smaller subsequence length
- Switch to Conservative mode
- Disable overlap temporarily

### 📁 **File Format Issues**  
**Problem**: FASTA not recognized or parsing errors  
**Solutions**:
- Ensure headers start with '>'
- Check for unusual characters
- Verify file not corrupted
- Try manual decompression

### 🧬 **Poor Gene Predictions**
**Problem**: Too many/few genes predicted  
**Solutions**:
- **Too many**: Increase peak-threshold (0.8-0.95)
- **Too few**: Decrease peak-threshold (0.6), reduce min-coding-length
- **Wrong lineage**: Verify model selection
- **Assembly quality**: Check if N50 is very low

### ⚡ **Slow Performance**
**Problem**: Prediction taking too long  
**Solutions**:
- Use Auto-Optimized mode
- Increase batch size if GPU memory allows
- Verify GPU is being used
- Use larger subsequence lengths for good assemblies

### 🔧 **Parameter Safety Violations**
**Problem**: Subsequence length warnings  
**Solutions**:
- Always use Auto-Optimized mode for safety
- Manual mode: ensure subsequence_length < 50% of N50
- Consider improving assembly quality
- Use Conservative mode for problematic assemblies

## Expected Performance by Assembly Quality

### 🚀 **Speed Improvements**
| Assembly Quality | Subsequence Length | Speed Gain | Safety |
|-----------------|-------------------|------------|---------|
| **Excellent** | Up to 320kb | 1.5-5x faster | ✅ Safe |
| **Good** | Up to 213kb | 1.7-3x faster | ✅ Safe |  
| **Fair** | Up to 106kb | 1-2x faster | ⚠️ Check N50 |
| **Poor** | Conservative | Baseline | ✅ Safe |

### 📊 **BUSCO Validation Expectations**
- **Excellent assemblies + Optimized params**: >95% complete
- **Good assemblies + Standard params**: 90-95% complete
- **Fair assemblies + Conservative params**: 85-90% complete
- **Poor assemblies**: <85% complete (assembly limitation)

## File Usage Guide

### 📁 **Output Files Explained**
- **`*_annotation.gff3`**: Standard GFF3 for most tools
- **`*_geneious.gff3`**: Optimized for Geneious/genome viewers
- **`*_proteins.fasta`**: For BUSCO, functional annotation
- **`*_transcripts.fasta`**: For RNA-seq analysis
- **`*_cds.fasta`**: For phylogenetics, codon usage
- **`*_genes.fasta`**: For promoter analysis
- **`*_summary.txt`**: Detailed statistics and metrics

### 🔍 **Genome Viewer Compatibility**
- **Geneious Prime**: Use `*_geneious.gff3`
- **IGV**: Use standard `*_annotation.gff3`
- **JBrowse**: Use standard `*_annotation.gff3`
- **UCSC Genome Browser**: May need format conversion

## Additional Resources

- **📚 GitHub**: https://github.com/weberlab-hhu/Helixer
- **📄 Paper**: https://doi.org/10.1101/2023.02.06.527280
- **🧬 BUSCO**: https://busco.ezlab.org/
- **🔧 gffread**: https://github.com/gpertea/gffread
- **📖 Parameter docs**: Full parameter list in GitHub README

## Getting Help

1. **Check assembly statistics** - Most issues relate to poor assembly quality
2. **Use Auto-Optimized mode** - Prevents most parameter problems
3. **Validate with BUSCO** - Quantifies annotation completeness
4. **Compare lineage models** - Try different models if results poor
5. **Report issues** with assembly stats and error logs